# Setup
For this project we need the torch, transformers and sentencepiece Python packages in order to load and use pre-trained models. We will also need the huggingface_hub package to programmatically login to the Hugging Face Hub and manage repositories, and the datasets package to download datasets from the Hub.

The sentencepiece package is required by transformers to perform inference with some of the pre-trained open source models on Hugging Face Hub and does not need to be explicitly imported. Import the remaining packages as follows.

Run the provided !pip code to install necessary packages and restart your kernel.
Import torch
Import huggingface_hub using the alias hf_hub.
Import datasets
Import transformers

In [23]:
# %pip install datasets
# %pip install huggingface_hub
# %pip install pyarrow
# %pip install transformers
# %pip install hf_xet

In [24]:
import os # Import os
import torch # Import torch
import huggingface_hub as hf_hub # Import huggingface_hub using the alias hf_hub
import datasets # Import datasets 
import transformers # Import transformers

# Downloading pre-trained models
- Hugging Face Hub is a Git-based platform hosting model and dataset repos (each with a README and examples, e.g., bert-base-cased).
- Use transformers or diffusers to load and run pretrained models.
- Auto classes unify loading across architectures:
    - AutoModel, AutoModelFor<TASK>, AutoTokenizer, AutoFeatureExtractor, AutoImageProcessor, AutoProcessor.
- from_pretrained(<repo_or_path>) downloads configs, weights, and preprocessors.
- In this task, load the “cardiffnlp/twitter-roberta-base-emoji” tokenizer and model—a tweet emoji classifier. from_pretrained() fetches the necessary tokenizer files from that repo.


In [36]:
# Import the AutoTokenizer and AutoModel classes from transformers
from transformers import AutoTokenizer, AutoModel

# Load the pre-trained tokenizer of the "cardiffnlp/twitter-roberta-base-emoji" model
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emoji")

What from_pretrained() does

The from_pretrained() method first searches for a model repository with the same name on the Hugging Face Hub but it also accepts a local path or a URL with the expected folder structure. You can simply git clone the repository and load it from your local path.

In [37]:
# Print the tokenizer to see the data preprocessing configuration for this model 
print(tokenizer)

RobertaTokenizerFast(name_or_path='cardiffnlp/twitter-roberta-base-emoji', vocab_size=50265, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}


Tokenizers

NLP models can't process text inputs as it is and need to be converted to a fixed length mathematical format. The Tokenizer classes preprocess the text such that each word and punctuation is given a unique ID or a token, short sentences are padded and long sentences are truncated to create fixed-size input vectors. They also allow using additional tokens such as bos, eos, unk, sep, and more to specify start and end of sentences, and to assign token IDS to unknown words that are not in the tokenizer vocabulary.

The output of the tokenizer tells us this pretrained model uses a tokenizer with a 50265 unique token IDs that applies padding or truncation to the end of each input text, and removes leading (leftside) extra whitespaces. The tokenizer also returns attention masks that tell the model which tokens are actual input tokens and which ones are just padding.

In [38]:
# Load the pre-trained model of the "cardiffnlp/twitter-roberta-base-emoji" model
model = AutoModel.from_pretrained("cardiffnlp/twitter-roberta-base-emoji")

Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emoji and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Notes on Model Loading

Take a look at the warning above. We were only able to load a chunk of the model parameters included in the checkpoint with the AutoModel class.

This is because all transformers models are designed to have a single base class and multiple task-specific prediction classes built on top of it. The AutoModel class is designed to only load the base model parameters such as RobertaModel but not task-specific models such as RobertaForSequenceClassification.

In order to identify the exact class name and task of your target checkpoint, you can simply refer to the model configuration.

In [28]:
# Get the model configuration 
config = model.config

# Print model config's architectures attribute to see the model class name
print(model.config.architectures)

['RobertaForSequenceClassification']


We found out that the pretrained model we are trying to load is an instance of the RobertaForSequenceClassification class.

Important Tip

You don't have to download and print out the configuration to identify the model class name. You can simply navigate to the 
config.json
 file within the model repository to find it out.

# Task 2: Downstream Tasks and Task-Specific AutoModel Classes
* Limitations of AutoModel

In the previous task, we learned all about Auto classes and the limitations of the AutoModel class. We also learned that each transformers model consists of a base class and one or more task-specific classes. For example, RoBERTa has the base RobertaModel class and multiple prediction classes such as RobertaForSequenceClassification, RobertaForQuestionAnswering, etc. that leverage the base class.

Task-Specific AutoModel Classes

Luckily for us, transformers has multiple task-specific AutoModelForTASK classes to load pretrained models. 
In this task, you will learn how to load a pretrained model with a task-specific Auto model class.

* Instructions
Load the 
cardiffnlp/twitter-roberta-base-emoji
 model we used in the previous task with the correct AutoModelForTASK class.

cardiffnlp/twitter-roberta-base-emoji is an instance of the RobertaForSequenceClassification class, take a look at the list of 
Auto Classes
Identify and import the correct Auto class
Load the pretrained model

In [29]:
# Load the correct AutoModel class for RobertaForSequenceClassification
from transformers import AutoModelForSequenceClassification

# Load the pretrained "cardiffnlp/twitter-roberta-base-emoji" model using the correct auto model class
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emoji")

# Task 3: Explicit Model and Preprocesser Class Names
We learned how to identify the data preprocessor and model class of a model repository on the Hub in the previous task. Note that you can also do this by navigating to the Files and versions tab on the repository page.

Similar to models with multiple task-specific model classes, data preprocessor classes are named based on the model name and the modality.

Tokenizer classes for language models
FeatureExtractor classes for audio models
ImageProcessor classes for computer vision models
Processor classes for multi-modal models
These data preprocessor classes transform raw input data such as text, image, etc. to the format expected by your target pre-trained model. For an vision model, the preprocessor performs steps such as resizing to a fixed size, normalization, cropping, etc. In this task, you will

Use the "cardiffnlp/twitter-roberta-base-emoji" model we loaded with Auto classes before Explicitly import a model and its preprocessor class
Use explicit class names to load a pretrained model and its preprocessor

Instructions
Start by explicitly importing the tokenizer and model class of "cardiffnlp/twitter-roberta-base-emoji", which we identified in our previous task.

Use the from_pretrained method to download and load the tokenizer and model

In [30]:
# Import the explicit tokenizer class of "cardiffnlp/twitter-roberta-base-emoji" and the explicit model class of "cardiffnlp/twitter-roberta-base-emoji"
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification

# Load the tokenizer and model using the from_pretrained method
tokenizer = RobertaTokenizerFast.from_pretrained("cardiffnlp/twitter-roberta-base-emoji")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emoji")

# Task 4: Creating an NLP Pipeline - Part 1/2
You learned how to navigate the Hugging Face Hub repositories, identify the model and data preprocessor classes of your target repository and how to import and load a pre-trained model and its corresponding preprocessor.

You will now put these skills to use and create powerful language translation pipeline with a few lines of code using the 
google/flan-t5-base
 model. This multi-lingual translation model allows you to translate between multiple languages. For a full list of languages this model supports, simply refer to its model repository.

Text-to-Text Generation

In transformers, NLP models with text input and output fall under the AutoModelForSeq2SeqLM class. This class of pretrained models perform tasks such as translation, question answering and text completion.

Inputs for T5

The tokenizer of T5 preprocesses the raw text input to return its token ids and attention mask. Each token id represents a unique word, tag or punctuation in the model's vocabulary. The attention mask consists of 1s and 0s for each token to decide whether that token should be attended to or ignored by the transformer model.

In this task, you will learn

What the tokenizer classes do under the hood
How to preprocess your raw input text

* Instructions

Use the AutoTokenizer and AutoModelForSeq2SeqLM classes to load the pre-trained "google/flan-t5-base" model and its tokenizer
Create a text input that includes
A prompt to specify source and target languages for translation
Text to be translated
Preprocess the text input to retrieve token ids and the attention mask

In [31]:
# Import AutoModelForSeq2SeqLM from transformers and Load the pre-trained google/flan-t5-base model and tokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

We are ready to perform inference with the pretrained model. We will start by defining an input sentence where we start by which language we would like to translate from and to, and add the sentence to be translated.

In [32]:
# Specify source and target languages for translation
source_lang = "English"
target_lang = "German"
input_text = f"translate {source_lang} to {target_lang}: How old are you?"

# Use the tokenizer to preprocess input text, with return_tensors="pt" to return PyTorch tensors 
inputs = tokenizer(input_text, return_tensors="pt") # "pt"=pytorch

# Print the key and values of the preprocessed inputs dictionary
for key, value in inputs.items():
    print(key, value)

input_ids tensor([[13959,  1566,    12,  2968,    10,   571,   625,    33,    25,    58,
             1]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


# Task 5: Creating an NLP Pipeline - Part 2/2
You preprocessed the input text and are ready learn how to perform inference. In this task, you will learn

How to use torch.no_grad() to perform inference fast and efficiently, and reduce memory usage
How to feed preprocessed inputs to the model
How to decode model output to make it human readable
Note that transformers and diffusers libraries support batched inputs, meaning you can inputs multiple text inputs or images, etc. as a list. Regardless of the number of inputs, the preprocess and model classes output batched results where each entry corresponds to an input.

Instructions
Using torch.no_grad()

Pass preprocessed inputs to the loaded model to perform inference
Save the model output to a variable named outputs
Decode the model output tokens to human readable format

In [39]:
# Use torch.no_grad() to prevent gradient accumulation - speeds up inference, reduces memory
with torch.no_grad():
    # Use model.generate method to perform inference
    # Use **inputs to pass the unpacked dictionary as multiple arguments to the generate method
    outputs = model.generate(**inputs)

# The output is a list of length batch_size - number of input texts - which is 1 in this case
# Print the length of outputs list
print(len(outputs))

TypeError: The current model class (RobertaModel) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'RobertaForCausalLM'}

The model output is a sequence of token IDs, where each token represents a unique word or punctuation in the vocabular. To decode it to a human readable format, we just need to map each token ID to its corresponding word. All Tokenizer classes of transformers provides a convenient decode method to do this.

In [34]:
# Use the decode method of the tokenizer to convert the output to a human readable format 
# print(tokenizer.decode(outputs[0]))
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
translated_text

'Wie old sind Sie?'

# Task 6: Datasets Library
The transformers library is designed to make it easy to load and use pre-trained models, and to preprocess raw input data. However, it is up to you to find or create interesting datasets.

We will now take a look at the datasets library and learn how to find, download and create datasets with a few lines of code. This library is part of the Hugging Face ecosystem, and similar to transformers and diffusers, is tightly integrated with the Hugging Face Hub. Each library is stored as a Git repository on the Hub and can be downloaded with a single line of code.

In this task, you will learn

How to use the load_dataset() function to download and load a dataset from the Hub
How to only download a specific data split (training, validation, test)
Instructions
Import the load_dataset() function from datasets
Load the "adirik/fashion_image_caption-100" dataset
Print the dataset to see what features it contains
Print a sample of the dataset
Download only the training split of the dataset

In [40]:
# Import load_dataset from datasets package
from datasets import load_dataset
import shutil
import os

# Clear the specific dataset cache
cache_dir = os.path.expanduser("~/.cache/huggingface/datasets/adirik___fashion_image_caption-100")
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
# Use the load_dataset function to load the "adirik/fashion_image_caption-100" dataset
# download_mode="force_redownload" will clear the cache and download fresh
dataset = load_dataset("adirik/fashion_image_caption-100", download_mode="force_redownload")
dataset = load_dataset("adirik/fashion_image_caption-100", cache_dir="./data_cache")
# Print the dataset object
print(dataset)

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 83.35it/s]s]
                                                                      

Dataset parquet downloaded and prepared to file://C:/Users/Tiya/.cache/huggingface/datasets/adirik___parquet/adirik--fashion_image_caption-100-522dc40e35adf87a/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

Each dataset is a dictionary with train, and optionally val and test splits. Each split contains an iterable subset of the data with one or more keys that represent the features of the dataset defined by the dataset owner. For example, the "adirik/fashion_image_caption-100" dataset contains image and text keys.

In [ ]:
# Print the fifth data sample in the dataset - an image and a text caption pair
print(dataset['train'][4])

NameError: name 'dataset' is not defined